# Выбор локации для скважины

Вы работаете в добывающей компании. Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
df_0 = pd.read_csv('C:/Users/Денис/Desktop/Проекты/neftedobicha/geo_data_0.csv')
df_1 = pd.read_csv('C:/Users/Денис/Desktop/Проекты/neftedobicha/geo_data_1.csv')
df_2 = pd.read_csv('C:/Users/Денис/Desktop/Проекты/neftedobicha/geo_data_2.csv')

df_0.info()
df_1.info()
df_2.info()

display(df_0.head())
display(df_1.head())
display(df_2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [3]:
df_0 = df_0.drop(['id'], axis=1)
df_1 = df_1.drop(['id'], axis=1)
df_2 = df_2.drop(['id'], axis=1)

display(df_0.head())
display(df_1.head())
display(df_2.head())

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


In [4]:
state = np.random.RandomState(12345)
pd.options.mode.chained_assignment = None

Загрузили данные, посмотрели на величины в таблицах, в принципе пропусков в значениях нет, что за признаки нам тоже не известно, столбец с id удалил в связи с тем, что в нем только буквенные значения, т.е. индификатор скважины, а значит в дальнейшем он нам ничем не поможет, в том числе и при обучении модели. (P.S. видел что в таблицах присутствуют несколько строк с одинаковым id, но разными остальными значениями, предположу, что это вероятнее всего одно добывающее предприятия на территории которого, либо рядом, расположены две различные скважины, т.к. это все таки различные скважины не стоит суммировать их значения( к тому же мы не знаем, что обозначают признаки), потому просто оставим строки как есть)

## Обучение и проверка модели

In [8]:
# создание функции

def preditor(df):
    
    df_train, df_valid = train_test_split(df, test_size=0.25, random_state=state)
    
    features_train = df_train.drop(['product'], axis=1)
    target_train = df_train['product']
    features_valid = df_valid.drop(['product'], axis=1)
    target_valid = df_valid['product']
    
    numeric=['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(features_train[numeric]) 
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])

    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    
    print('Средний запас сырья реальный:', target_valid.mean())
    print('Средний запас сырья предсказанный:', predictions.mean())
    print('RMSE:', (mean_squared_error(target_valid, predictions) ** 0.5))

    df_valid['predict'] = predictions
    df_valid_pred = df_valid.reset_index(drop=True)
    
    return df_valid_pred

In [9]:
#для нулевого региона
df_valid_pred_0 = preditor(df_0)
df_valid_pred_0.head()

Средний запас сырья реальный: 92.00127344402844
Средний запас сырья предсказанный: 92.40129321593406
RMSE: 37.91718878682678


,f0,f1,f2,product,predict
0,-0.115431,0.971400,0.951573,58.498891,69.767622
1,0.472449,-0.207866,-1.015091,36.790923,75.592468
2,1.768375,-0.017880,3.675964,115.607489,108.927138
3,0.927104,0.257296,4.011032,87.088342,104.120214
4,0.265393,-0.163707,2.978482,33.397142,100.753785


In [10]:
#для первого
df_valid_pred_1 = preditor(df_1)
df_valid_pred_1.head()

Средний запас сырья реальный: 69.23037704592916
Средний запас сырья предсказанный: 69.24215404831706
RMSE: 0.8936594805071


,f0,f1,f2,product,predict
0,10.590292,-4.430363,0.994404,26.953261,27.022081
1,16.037336,-10.861428,-0.002844,0.000000,-0.502899
2,14.110586,-1.563653,4.993929,134.766305,134.240537
3,-6.776864,-8.852209,0.002228,3.179103,2.892721
4,-11.096395,-3.620638,0.992688,30.132364,30.097661


In [11]:
#и второго
df_valid_pred_2 = preditor(df_2)
df_valid_pred_2.head()

Средний запас сырья реальный: 95.03996726290791
Средний запас сырья предсказанный: 94.79275536960412
RMSE: 40.206897848340155


,f0,f1,f2,product,predict
0,2.116811,2.464209,2.543555,136.847051,95.098687
1,-1.277833,0.019428,5.025745,124.297699,109.610468
2,-2.743839,-3.458149,2.605033,90.306873,95.728762
3,-0.493303,0.113255,-4.558502,54.934203,54.242082
4,-0.467235,0.497568,-4.283809,112.636028,55.824327


Мы получили довольно интересные результаты:
  1. Во-первых, стоит отметить средние предсказанные значения для нулевого и второго регионов, которые значительно выше, чем у первого и прктически совпадают между собой.
  2. Во-вторых, несмотря на то, что у первого региона средний запас предсказанного сырья практически на четверть меньше, этот регион более стабильный, практически все скважены будет давать нам одно и то же количество сырья, в отличие от других, в которых будут уже риски.

## Подготовка к расчёту прибыли

По условию:

 - бюджет разработки скважин = 10 000 000 000
 - прибыль с 1 ед. продукта = 450 000
 - количество иследуемых точек = 500
 - в разроботку попадают 200 лучших
 
Тогда для того, чтобы окупить бюджет мы должны с 200 лучших точек получить 22223 единицы продукта.


Проверим по нашим регионам. Для этого отберем 200 лучших точек по предсказаниям модели, а затем сложим фактические результаты и сравним с количеством продукта, требуемым для окупа бюджета.

In [14]:
reg_0 = df_valid_pred_0.sort_values(by='predict',ascending=False).head(200)
reg_0[:5]

,f0,f1,f2,product,predict
17719,1.888221,0.067384,15.428372,144.009283,186.323583
6940,1.853784,-0.153503,13.585450,153.639837,177.069557
5244,1.412823,-0.612153,12.784117,162.153488,176.597070
458,1.958217,0.239926,14.279737,96.893581,176.500252
6052,1.549533,-0.202918,13.518043,116.045820,176.187668


In [15]:
reg_0['product'].sum()

29481.53070322214

In [16]:
def scaler(df):
    reg = df.sort_values(by='predict',ascending=False).head(200)
    result = reg['product'].sum()
    return result

In [17]:
scaler(df_valid_pred_0)

29481.53070322214

In [18]:
scaler(df_valid_pred_1)

27589.081548181137

In [19]:
scaler(df_valid_pred_2)

27400.804009555275

Все три региона дают явно больше прибыли со своих лучших месторождений

## Расчёт прибыли и рисков 

Воспользуемся техникой бутстреп.

In [22]:
target = df_valid_pred_0['product']
probabilities = df_valid_pred_0['predict']

def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return 450000 * selected.sum() - 10000000000

values = []
for i in range(1000):
    target_subsample = target.sample(n=500, replace=True, random_state=state)
    probs_subsample = probabilities[target_subsample.index]
 
    values.append(revenue(target_subsample, probs_subsample, 200))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

mean = values.mean()
print("Средняя прибыль:", mean)
print("2.5%-квантиль:", lower)
print("97.5%-квантиль:", upper)
print(target.shape)

Средняя прибыль: 346659076.67169875
2.5%-квантиль: -180028724.7309289
97.5%-квантиль: 893435930.5634305
(25000,)


In [23]:
# риск рассчитаем как среднее по отрицательным значениям валуе, и умножим на 100, чтобы увидеть данную величину в процентах.
(values < 0).mean()*100


9.0

Средняя выручка для нулевого региона 425 063 856. Однако судя по полученным результатам 2,5%-квантиля, шанс того, что мы не только не получим прибыль, но еще и уйдем в минус будет составлять явно больше, чем 2,5%

По аналогии рассчитаем прибыль для первого региона.

In [24]:
target = df_valid_pred_1['product']
probabilities = df_valid_pred_1['predict']

def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return 450000 * selected.sum() - 10000000000

values = []
for i in range(1000):
    target_subsample = target.sample(n=500, replace=True, random_state=state)
    probs_subsample = probabilities[target_subsample.index]
 
    values.append(revenue(target_subsample, probs_subsample, 200))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

mean = values.mean()
print("Средняя прибыль:", mean)
print("2.5%-квантиль:", lower)
print("97.5%-квантиль:", upper)

Средняя прибыль: 549734661.8655714
2.5%-квантиль: 145955923.5971037
97.5%-квантиль: 998487051.6927644


Средняя выручка для первого региона 501 691 386. Причем риск того, что мы неокупим бюджет, для данного региона, судя по доверительному интервалу будет составлять менее 2,5%.

In [25]:
# аналогично для первого региона
(values < 0).mean()*100

0.3

И для второго:

In [26]:
target = df_valid_pred_2['product']
probabilities = df_valid_pred_2['predict']

def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return 450000 * selected.sum() - 10000000000

values = []
for i in range(1000):
    target_subsample = target.sample(n=500, replace=True, random_state=state)
    probs_subsample = probabilities[target_subsample.index]
 
    values.append(revenue(target_subsample, probs_subsample, 200))

values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

mean = values.mean()
print("Средняя прибыль:", mean)
print("2.5%-квантиль:", lower)
print("97.5%-квантиль:", upper)

Средняя прибыль: 388140689.1655275
2.5%-квантиль: -143870117.94809535
97.5%-квантиль: 921161435.2642229


Средняя выручка для первого региона 363 190 605, тогда с учетом бюджета на разработку, средняя прибыль с региона будет сотавлять 367 048 310. Однако и в этом регионе шанс не окупить бюджет составляет более 2,5%.

In [27]:
# и для второго
(values < 0).mean()*100

8.9

## Вывод

В ходе работы были заргужены и подготоленные данные по трем регионам, в которых ведеться добыча нефти. Для каждого региона была обучена и проверена модель, предсказывающая количество единиц продуктва в скважинах. Но основе данной содели было проведено исследование для определения региона, гдедобыча принесет больше прибыли. Техникой бутсртеп проанализированна и полученна возможная прибыль, а также возможные риски. В качестве итогов, для разработки региона с целью получения прибыли стоит порекомендовать первый регион, который с рисками менее 2,5% принесет в среднем прибыль равную 515 704 525.